In [2]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re
import os
import shutil
import mysql.connector

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from time import sleep
from datetime import datetime

In [3]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')

## Encontrar links nuevos y subirlos a la bbdd

In [4]:
contraseña_miguel = "password"
contraseña_moran = 'Deportivo96'
contraseña_carlos = "Whereisbotus1996"

In [5]:
# Funcion para extraer datos
def extraer_datos(nombre_tabla, columna = '*'):
    
    try:
        # Conexión a la base de datos de MySQL
        conn = mysql.connector.connect(
            host = "localhost",
            user = "root",
            password = contraseña_moran,
            database = "coches_segunda_mano"
        )
        # Crear un cursor para ejecutar comandos SQL
        cursor = conn.cursor()

        # Crear la consulta SQL para seleccionar todos los datos de la tabla
        query = f"SELECT {columna} FROM {nombre_tabla}"

        # Ejecutar la consulta y recuperar los datos
        cursor.execute(query)
        resultado = cursor.fetchall()

        # Obtener los nombres de las columnas de la tabla
        columnas = [desc[0] for desc in cursor.description]

        # Convertir los resultados a un DataFrame
        df = pd.DataFrame(resultado, columns=columnas)

        # Cerrar el cursor y la conexión
        cursor.close()
        conn.close()

        # Devolver el DataFrame
        print(f"Datos de {nombre_tabla} extraidos correctamente")
        return df
    
    except mysql.connector.Error as e:
        print(f"Error al conectar con la base de datos: {e}")
        return pd.DataFrame() 
    
    finally:
        # Asegurarse de cerrar la conexión y el cursor
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

In [6]:
df_lista_coches = extraer_datos('links_coches')
enlaces_antiguos = set(df_lista_coches['link_anuncio'])

Datos de links_coches extraidos correctamente


In [7]:
url = f"https://www.autocasion.com/coches-ocasion"
browser = webdriver.Chrome()
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

browser.quit()

print(f"Total de coches: {total_coches}")
print(f"Total de páginas: {total_paginas}")

Total de coches: 105503
Total de páginas: 6207


In [8]:
data = []
total_anuncios = set()
browser = webdriver.Chrome()

try:
    #detener_bucle = False
    
    for i in tqdm.tqdm(range(1, total_paginas + 1)):
        if detener_bucle:
            break
            
        url = f"https://www.autocasion.com/coches-ocasion?page={i}&direction=desc&sort=updated_at"
        browser.get(url)
        sleep(random.uniform(2, 5))
        
        soup = BeautifulSoup(browser.page_source, "html.parser")
        coches_disponibles = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.', ''))
        
        if coches_disponibles == 0:
            break
        
        # Encontrar todos los anuncios en la página
        anuncios_pagina = soup.find_all("article", class_="anuncio")

        for anuncio in anuncios_pagina:
            try:
                
                link_anuncio = anuncio.find("button", class_="copiar")["rel"]

                # Si el anuncio ya está en los antiguos, detener el bucle
                # if link_anuncio in enlaces_antiguos:
                #     print(f"Anuncio ya existente encontrado: {link_anuncio}. Deteniendo el bucle.")
                #     detener_bucle = True
                #     break

                # Si el anuncio ya fue procesado en esta extracción, omitir
                if link_anuncio in total_anuncios:
                    continue

                total_anuncios.add(link_anuncio)

                # Extraer y procesar la foto
                link_foto = anuncio.find("img")["src"]
                response_img = requests.get(link_foto, stream=True)
                response_img.raise_for_status()
                foto_binaria = response_img.content

                data.append({
                    "link_anuncio": link_anuncio,
                    "foto_binaria": foto_binaria
                })

            except Exception as e:
                print(f"Error procesando un anuncio: {e}")
                continue
finally:
    browser.quit()
    
print(f"Anuncios totales únicos: {len(total_anuncios)}")

  0%|          | 1/6207 [00:10<17:57:08, 10.41s/it]

Anuncio ya existente encontrado: https://www.autocasion.com/coches-segunda-mano/citroen-c3-ocasion/c3-1-2-puretech-s-s-shine-83-ref14865164. Deteniendo el bucle.


Anuncios totales únicos: 8


In [9]:
enlaces_nuevos = total_anuncios - enlaces_antiguos
print(f"Enlaces nuevos añadidos: {len(enlaces_nuevos)}")

Enlaces nuevos añadidos: 8


In [10]:
index = df_lista_coches['id_coche'].max()+ 1 if not df_lista_coches.empty else 1

df_links = pd.DataFrame(data)
df_nuevos_links = df_links[df_links['link_anuncio'].isin(enlaces_nuevos)]
df_nuevos_links = df_nuevos_links.reset_index(drop=True)
df_nuevos_links['id_coche'] = range(index, index + len(df_nuevos_links))
df_nuevos_links = df_nuevos_links[['id_coche', 'link_anuncio', 'foto_binaria']]

In [11]:
def insertar_datos_en_lotes(df, nombre_tabla, lote=100):
    # Conexión a la base de datos MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password=contraseña_moran,
        database="coches_segunda_mano"
    )
    cursor = conn.cursor()

    columnas = df.columns.tolist()
    query = f"""
        INSERT INTO {nombre_tabla} ({', '.join(columnas)})
        VALUES ({', '.join(['%s'] * len(columnas))})
    """

    # Dividir el DataFrame en lotes
    for i in range(0, len(df), lote):
        valores = [tuple(row) for row in df.iloc[i:i + lote].to_numpy()]
        cursor.executemany(query, valores)
        conn.commit()

    cursor.close()
    conn.close()
    print(f"Datos insertados en {nombre_tabla} correctamente.")

In [12]:
insertar_datos_en_lotes(df_nuevos_links, 'links_coches')

Datos insertados en links_coches correctamente.


## Scraping links nuevos

In [14]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [15]:
coches_df = dict()
browser = webdriver.Chrome()

for coche in tqdm.tqdm(df_nuevos_links['link_anuncio']):
    #print(coche)
    
    try: 
        url_coche = coche
        browser.get(url_coche)
        soup = BeautifulSoup(browser.page_source)
        
    except: 
        continue
        
    
    try:
        if 'Vehículo vendido' in soup.find('div', class_= 'img-ficha').find('p').text.replace('X',''):
            continue
            
    except: 
        
        pass
    
    info_coche = {}
            
    try: 
        info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
        
    except: 
        info_coche['provincia'] = 'no disponible'
        
    info_coche['link_coche'] = coche
    info_coche['fecha_extraccion'] = datetime.now().strftime("%d/%m/%Y")
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
        
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        try: 
            detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
            for detalle in detalles_carroceria:
                detalle = detalle.text.replace('Medidas','').strip()

                if 'Largo' in detalle: 
                    info_coche['largo'] = detalle
                elif 'Ancho' in detalle: 
                    info_coche['ancho'] = detalle
                elif 'Alto' in detalle: 
                    info_coche['alto'] = detalle
                #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
                elif 'Capacidad del maletero' in detalle: 
                    info_coche['capacidad_maletero'] = detalle
                elif 'Carrocería' in detalle: 
                    info_coche['carroceria'] = detalle
                elif 'Número de plazas' in detalle: 
                    info_coche['num_plazas'] = detalle
                elif 'Número de puertas' in detalle: 
                    info_coche['num_puertas'] = detalle
                elif 'Batalla' in detalle: 
                    info_coche['batalla'] = detalle
                #supongo que si hay dos pesos el segundo será la masa max. autorizada
                elif 'Peso' in detalle: 
                    info_coche['peso'] = detalle
                    
        except:   
            info_coche['largo'] = 'no disponible'
            info_coche['ancho'] = 'no disponible'
            info_coche['alto'] = 'no disponible'
            info_coche['capacidad_maletero'] = 'no disponible'
            info_coche['carroceria'] = 'no disponible'
            info_coche['num_plazas'] = 'no disponible'
            info_coche['num_puertas'] = 'no disponible'
            info_coche['batalla'] = 'no disponible'
            info_coche['peso'] = 'no disponible'
                
           
        try:
            detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
            for detalle in detalles_consumos:
                detalle = ' '.join(detalle.text.replace('\n','').split())

                if 'Consumo medio' in detalle: 
                    info_coche['consumo_medio'] = detalle
                elif 'Consumo en carretera' in detalle: 
                    info_coche['consumo_carretera'] = detalle
                elif 'Consumo urbano' in detalle: 
                    info_coche['consumo_urbano'] = detalle
                elif 'CO2' in detalle: 
                    info_coche['CO2'] = detalle
                elif 'Depósito' in detalle: 
                    info_coche['deposito'] = detalle
                    
        except: 
            info_coche['consumo_medio'] = 'no disponible'
            info_coche['consumo_carretera'] = 'no disponible'
            info_coche['consumo_urbano'] = 'no disponible'
            info_coche['CO2'] = 'no disponible'
            info_coche['deposito'] = 'no disponible'        
        
        try:
            detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
            for detalle in detalles_motor:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Combustible' in detalle: 
                    info_coche['combustible'] = detalle
                elif 'Nº de cilindros' in detalle: 
                    info_coche['cilindros'] = detalle
                elif 'Cilindrada' in detalle: 
                    info_coche['cilindrada'] = detalle
                elif 'Sobrealimentación' in detalle: 
                    info_coche['sobrealimentacion'] = detalle
                    
        except: 
            info_coche['combustible'] = 'no disponible'
            info_coche['cilindros'] = 'no disponible'
            info_coche['cilindrada'] = 'no disponible'
            info_coche['sobrealimentacion'] = 'no disponible'

        try:
            detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
            for detalle in detalles_transmision:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Tipo de transmisión' in detalle: 
                    info_coche['tipo_cambio'] = detalle
                elif 'Tracción' in detalle: 
                    info_coche['traccion'] = detalle
                elif 'Número de marchas' in detalle: 
                    info_coche['num_marchas'] = detalle
                    
        except: 
            info_coche['tipo_cambio'] = 'no disponible'
            info_coche['traccion'] = 'no disponible'
            info_coche['num_marchas'] = 'no disponible'

        try:
            detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
            for detalle in detalles_prestaciones:
                detalle = detalle.text

                if 'Potencia (KW)' in detalle: 
                    info_coche['potencia_kw'] = detalle
                elif 'Potencia (CV)' in detalle: 
                    info_coche['potencia_cv'] = detalle
                elif 'Par' in detalle: 
                    info_coche['par'] = detalle
                elif 'Velocidad' in detalle: 
                    info_coche['velocidad_max'] = detalle
                elif 'Aceleración' in detalle: 
                    info_coche['aceleracion'] = detalle
                    
        except: 
            info_coche['potencia_kw'] = 'no disponible'
            info_coche['potencia_cv'] = 'no disponible'
            info_coche['par'] = 'no disponible'
            info_coche['velocidad_max'] = 'no disponible'
            info_coche['aceleracion'] = 'no disponible'
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(3, 6))
    
browser.quit()

100%|██████████| 8/8 [01:18<00:00,  9.84s/it]


In [16]:
df = pd.DataFrame.from_dict(coches_df).T

## Scraping marcas

In [18]:
tabla_marcas = extraer_datos('marcas', columna = '*')
marcas_guardadas = set(tabla_marcas['nombre_marca'])

Datos de marcas extraidos correctamente


In [19]:
browser = webdriver.Chrome()
url = 'https://www.autocasion.com/marcas'
browser.get(url)
soup = BeautifulSoup(browser.page_source)
marcas = soup.find('ul', class_= 'marcas').find_all('span')


marcas_nuevas = {}
index = tabla_marcas['id_marca'].max() + 1 if not tabla_marcas.empty else 1
for marca in marcas:
    marca = marca.text.strip()
    
    if marca not in marcas_guardadas:
    
        marcas_nuevas[index] = marca
    
browser.quit()

df_nuevas_marcas = pd.DataFrame.from_dict(marcas_nuevas, orient='index', columns=['nombre_marca']).reset_index()
df_nuevas_marcas.rename(columns={'index': 'id_marca'}, inplace=True)

In [20]:
if not df_nuevas_marcas.empty:
    
    insertar_datos_en_lotes(df_nuevas_marcas, 'provincias', lote=100)
    
    print(f"{len(df_nuevas_marcas)} marcas nuevas insertadas.")
else:
    print("No hay nuevas marcas para insertar.")

No hay nuevas marcas para insertar.


## Limpiar datos nuevos

In [22]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
19,peso,6,25.0,25.0,object
14,alto,6,25.0,25.0,object
36,velocidad_max,6,25.0,25.0,object
35,par,6,25.0,25.0,object
33,potencia_kw,6,25.0,25.0,object
32,num_marchas,3,25.0,50.0,object
31,traccion,2,25.0,62.5,object
29,sobrealimentacion,3,25.0,37.5,object
27,cilindros,3,25.0,50.0,object
25,deposito,6,25.0,25.0,object


In [23]:
# Sólo nos interesan vendedores profesionales, los particulares representan un % mínimo y cubren los datos mínimos

df = df[df["tipo_vendedor"] == "Profesional"]
df = df.drop(columns=['tipo_vendedor'])

In [24]:
#Hay nans que se toman como floats. Lo convierto a "no disponible"
df['tipo_cambio'] = df['tipo_cambio'].fillna("no disponible")

#No sé qué es manual automatizada, si queremos que sea manual hay que poner la lambda al revés
df['tipo_cambio'] = df['tipo_cambio'].apply(lambda x: 'Manual' if 'Manual' in x else 'Automático')

In [25]:
df["carroceria"] = df["carroceria"].str.replace("Carrocería","")

df["combustible"] = df["combustible"].str.replace("combustible ","").str.replace("Combustible ","")

In [26]:
df['fecha_matriculacion'] = df['fecha_matriculacion'].str.split("Matriculado: ").str[1]

def limpiar_fecha(fecha):
    try:
        mes, ano = fecha.split('/')
        return int(mes), int(ano)
    except:
        return np.nan, int(fecha)

df[['mes_matriculacion', 'ano_matriculacion']] = df['fecha_matriculacion'].apply(lambda x: pd.Series(limpiar_fecha(x)))

df = df.drop(columns=['fecha_matriculacion'])

In [27]:
df['mes_matriculacion'] = df['mes_matriculacion'].fillna(0).astype('int64')
df['ano_matriculacion'] = df['ano_matriculacion'].fillna(0).astype('int64')

In [28]:
df["garantia"] = df["garantia"].str.split(' ').str[0].replace("Sí",np.nan).replace("No",0).astype(float)

In [29]:
df["precio_contado"] = pd.to_numeric(df["precio_contado"], errors='coerce')

df["precio_nuevo"] = pd.to_numeric(df["precio_nuevo"], errors='coerce')

df['largo'] = pd.to_numeric(df['largo'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['ancho'] = pd.to_numeric(df['ancho'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['alto'] = pd.to_numeric(df['alto'].str.extract('(\d+\.?\d*)')[0], errors='coerce')

df['capacidad_maletero'] = pd.to_numeric(df['capacidad_maletero'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
df["num_plazas"] = df["num_plazas"].str.split("s").str[1].astype("Int64")
df["batalla"] = (df["batalla"].str.replace(r'Batalla \(mm\)\s*', '', regex=True).str.replace('.', '', regex=False))
df["batalla"] = pd.to_numeric(df["batalla"], errors='coerce').astype("Int64")
df["peso"] = df["peso"].str.replace('Peso/Masa max. autorizado (kg)', '').str.replace('.','').astype("Int64")
df["num_puertas"] = df["num_puertas"].str.extract(r'(\d+)').astype("Int64")
df["consumo_medio"] = df["consumo_medio"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_carretera"] = df["consumo_carretera"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_urbano"] = df["consumo_urbano"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["co2"] = df["CO2"].str.extract(r'(\d{1,3})(?= g/km)')[0].astype(float)
df["num_cilindros"] = df["cilindros"].str.extract(r'(\d+)').astype("Int64")
df["cilindrada"] = df["cilindrada"].str.extract(r'(\d+,\d+|\d+\.\d+)', expand=False).str.replace(',', '').str.replace('.', '').astype("Int64")
df["deposito"] = df["deposito"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

df["num_marchas"] = df["num_marchas"].str.extract(r'(\d+)').astype("Int64")
df["potencia_kw"] = df["potencia_kw"].str.extract(r'(\d+)').astype("Int64")
df["potencia_cv"] = df["potencia_cv"].str.extract(r'(\d+)').astype("Int64")
df["par"] = df["par"].str.extract(r'(\d+)').astype("Int64")
df["velocidad_max"] = df["velocidad_max"].str.extract(r'(\d+)').astype("Int64")
df["aceleracion"] = df["aceleracion"].str.extract(r'(\d+,\d+|\d+\.\d+)')[0].str.replace(",", ".").astype(float)
df['id_sobrealimentacion'] = df['sobrealimentacion'].apply(lambda x: x.replace('Sobrealimentación', '').replace('Tipo de sobrealimentador', 'no disponible').strip() if pd.notna(x) and x != '-' else 'no disponible')

In [30]:
df["traccion"] = df['traccion'].fillna("no disponible")
df["traccion"] = df["traccion"].str.replace('Tracción', '').str.replace('Todo terreno tracción', '').apply(lambda x: x.strip())

In [31]:
#marca_listado = extraer_datos('marcas', columna = 'nombre_marca')

lista_marcas = [marca.upper() for marca in marcas_guardadas]

df['marca_modelo'] = df['marca_modelo'].fillna('').str.replace("Detalles ", "").str.strip().str.upper()

df['marca'] = df['marca_modelo'].apply(lambda x: next((marca for marca in lista_marcas if marca in x), np.nan))
df['modelo'] = df.apply(lambda x: re.sub(r'\s*\(.*?\)', '', x['marca_modelo'].replace(x['marca'], "").strip()).strip() if pd.notnull(x['marca']) else x['marca_modelo'], axis=1)

In [32]:
df = df.drop(columns=['marca_modelo', 'CO2', 'cilindros', 'sobrealimentacion', 'carroceria'])

In [33]:
df.rename(columns={'distintivo_ambiental': 'id_distintivo_ambiental'
                  , 'marca': 'id_marca'
                  , 'modelo': 'id_modelo'
                  , 'nombre_vendedor': 'id_concesionario'
                  , 'provincia': 'id_provincia'
                  , 'traccion': 'id_traccion'}, inplace=True)

In [34]:
df['combustible'] = (
    df['combustible']
    .str.replace('-', 'no disponible')
    .str.replace('Diésel', 'Diesel')
    .str.replace('Gasolina y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Diesel y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Corriente eléctrica', 'Eléctrico', regex=False)
    .str.replace(r'\bHíbrido\b(?!.*Enchufable)', 'Híbrido Enchufable', regex=True)
)

In [35]:
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split('-')[0])
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split(',')[0])

In [36]:
for columna in df.columns:
    df[columna] = df[columna].replace(["no disponible", "<NA>", "-", "0", 0, np.nan], None)

In [37]:
df = df[df['precio_contado'].notna()]

In [38]:
df.sample(5)

,id_provincia,link_coche,fecha_extraccion,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
6,Barcelona,https://www.autocasion.com/coches-segunda-mano...,30/11/2024,12222,BYmyCAR,C,24.0,21910,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,None,Manual,None,None,None,100,None,None,None,7,2024,None,None,None,DACIA,JOGGER
5,Barcelona,https://www.autocasion.com/coches-km0/km-0/dac...,30/11/2024,5,BYmyCAR,0 EMISIONES,24.0,21111,21100.0,3734.0,1579.0,1516.0,270.0,4,2423,1300,5,None,None,None,None,Eléctrico,None,Automático,delantera,1,48,65,113,125,13.7,4,2024,None,None,None,DACIA,SPRING
2,Madrid,https://www.autocasion.com/coches-segunda-mano...,30/11/2024,18638,F. Tomé,ECO,24.0,43750,60031.0,4689.0,1893.0,1660.0,510.0,5,2819,2420,5,4.7,4.2,5.4,70.0,Diesel,1968,Automático,delantera,7,120,163,370,213,9.0,2,2022,123.0,4,Turbo de geometría variable,AUDI,Q5
0,Sevilla,https://www.autocasion.com/coches-segunda-mano...,30/11/2024,92000,AUTOCASION STADIUM,C,12.0,13500,22050.0,4202.0,1798.0,1590.0,423.0,5,2600,1920,5,4.3,3.9,5.1,47.0,Diesel,1597,Manual,delantera,6,85,115,300,175,None,4,2018,131.0,4,Turbo de geometría variable,KGM / SSANGYONG,TIVOLI
3,Barcelona,https://www.autocasion.com/coches-segunda-mano...,30/11/2024,18796,BYmyCAR,C,24.0,15900,16220.0,4099.0,1848.0,1535.0,328.0,5,2604,1578,5,4.9,4.3,5.9,50.0,Gasolina,None,Manual,delantera,6,67,91,97,172,12.0,6,2023,115.0,3,Turbo,DACIA,SANDERO


## Actualizar tablas de dimension

### Tabla provincias

In [41]:
tabla_provincias = extraer_datos('provincias', columna = '*')
provincias_guardadas = set(tabla_provincias['nombre_provincia'])

Datos de provincias extraidos correctamente


In [42]:
comunidades_dict = {
    "Andalucía": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"],
    "Aragón": ["Huesca", "Teruel", "Zaragoza"],
    "Asturias": ["Asturias"],
    "Islas Baleares": ["Islas Baleares"],
    "Canarias": ["Las Palmas", "Tenerife"],
    "Cantabria": ["Cantabria"],
    "Castilla-La Mancha": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"],
    "Castilla y León": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"],
    "Cataluña": ["Barcelona", "Girona", "Lleida", "Tarragona"],
    "Extremadura": ["Badajoz", "Cáceres"],
    "Galicia": ["La Coruña", "Lugo", "Orense", "Pontevedra"],
    "Madrid": ["Madrid"],
    "Murcia": ["Murcia"],
    "Navarra": ["Navarra"],
    "La Rioja": ["La Rioja"],
    "País Vasco": ["Álava", "Guipúzcoa", "Vizcaya"],
    "Valencia": ["Alicante", "Castellón", "Valencia"],
    "Ceuta": ["Ceuta"],
    "Melilla": ["Melilla"]
}

In [43]:
provincias_to_comunidades = {
    provincia: comunidad
    for comunidad, provincias in comunidades_dict.items()
    for provincia in provincias
}

In [44]:
nuevas_provincias = {}
index = tabla_provincias['id_provincia'].max() + 1 if not tabla_provincias.empty else 1

for provincia in df['id_provincia'].unique():
    if pd.notnull(provincia) and provincia not in provincias_guardadas: 
        nuevas_provincias[index] = provincia
        index += 1

df_nuevas_provincias = pd.DataFrame.from_dict(nuevas_provincias, orient='index', columns=['nombre_provincia']).reset_index()
df_nuevas_provincias.rename(columns={'index': 'id_provincia'}, inplace=True)
df_nuevas_provincias['comunidad_autonoma'] = df_nuevas_provincias['nombre_provincia'].map(provincias_to_comunidades)
df_provincias_actualizado = pd.concat([tabla_provincias, df_nuevas_provincias], ignore_index=True)

conversion = dict(zip(df_provincias_actualizado['nombre_provincia'], df_provincias_actualizado['id_provincia']))
df['id_provincia'] = df['id_provincia'].map(conversion)

In [45]:
if not df_nuevas_provincias.empty:
    
    insertar_datos_en_lotes(df_nuevas_provincias, 'provincias', lote=100)
    
    print(f"{len(df_nuevas_provincias)} provincias nuevas insertadas en la base de datos.")
else:
    print("No hay nuevas provincias para insertar.")

No hay nuevas provincias para insertar.


### Tabla concesionarios

In [47]:
tabla_concesionarios = extraer_datos('concesionarios', columna = '*')
concesionarios_guardados = set(tabla_concesionarios['nombre_concesionario'])

Datos de concesionarios extraidos correctamente


In [48]:
nuevos_concesionarios = {}
index = tabla_concesionarios['id_concesionario'].max() + 1 if not tabla_concesionarios.empty else 1

for concesionario in df['id_concesionario'].unique():
    if pd.notnull(concesionario) and concesionario not in concesionarios_guardados: 
        nuevos_concesionarios[index] = concesionario
        index += 1

df_nuevos_concesionarios = pd.DataFrame.from_dict(nuevos_concesionarios, orient='index', columns=['nombre_concesionario']).reset_index()
df_nuevos_concesionarios.rename(columns={'index': 'id_concesionario'}, inplace=True)

df_concesionarios_actualizado = pd.concat([tabla_concesionarios, df_nuevos_concesionarios], ignore_index=True)

conversion = dict(zip(df_concesionarios_actualizado['nombre_concesionario'], df_concesionarios_actualizado['id_concesionario']))
df['id_concesionario'] = df['id_concesionario'].map(conversion)

In [49]:
if not df_nuevos_concesionarios.empty:
    
    insertar_datos_en_lotes(df_nuevos_concesionarios, 'concesionarios', lote=100)
    
    print(f"{len(df_nuevos_concesionarios)} concesionarios nuevos insertados en la base de datos.")
else:
    print("No hay nuevos concesionarios para insertar.")

Datos insertados en concesionarios correctamente.
2 concesionarios nuevos insertados en la base de datos.


### Tabla modelos

In [51]:
tabla_modelos = extraer_datos('modelos', columna = '*')
modelos_guardados = set(tabla_modelos['nombre_modelo'])

Datos de modelos extraidos correctamente


In [52]:
nuevos_modelos = {}
index = tabla_modelos['id_modelo'].max() + 1 if not tabla_modelos.empty else 1

for modelo in df['id_modelo'].unique():
    if pd.notnull(modelo) and modelo not in modelos_guardados: 
        nuevos_modelos[index] = modelo
        index += 1

df_nuevos_modelos = pd.DataFrame.from_dict(nuevos_modelos, orient='index', columns=['nombre_modelo']).reset_index()
df_nuevos_modelos.rename(columns={'index': 'id_modelo'}, inplace=True)

df_modelos_actualizado = pd.concat([tabla_modelos, df_nuevos_modelos], ignore_index=True)

conversion = dict(zip(df_modelos_actualizado['nombre_modelo'], df_modelos_actualizado['id_modelo']))
df['id_modelo'] = df['id_modelo'].map(conversion)

In [53]:
if not df_nuevos_modelos.empty:
    
    insertar_datos_en_lotes(df_nuevos_modelos, 'modelos', lote=100)
    
    print(f"{len(df_nuevos_modelos)} modelos nuevos insertados en la base de datos.")
else:
    print("No hay nuevos modelos para insertar.")

No hay nuevos modelos para insertar.


### Tabla distintivos_ambientales

In [55]:
tabla_distintivos = extraer_datos('distintivos_ambientales', columna = '*')
distintivos_guardados = set(tabla_distintivos['nombre_distintivo'])

Datos de distintivos_ambientales extraidos correctamente


In [56]:
nuevos_distintivos = {}
index = tabla_distintivos['id_distintivo'].max() + 1 if not tabla_distintivos.empty else 1

for distintivo in df['id_distintivo_ambiental'].unique():
    if pd.notnull(distintivo) and distintivo not in distintivos_guardados: 
        nuevos_distintivos[index] = distintivo
        index += 1

df_nuevos_distintivos = pd.DataFrame.from_dict(nuevos_distintivos, orient='index', columns=['nombre_distintivo']).reset_index()
df_nuevos_distintivos.rename(columns={'index': 'id_distintivo'}, inplace=True)

df_distintivos_actualizado = pd.concat([tabla_distintivos, df_nuevos_distintivos], ignore_index=True)

conversion = dict(zip(df_distintivos_actualizado['nombre_distintivo'], df_distintivos_actualizado['id_distintivo']))
df['id_distintivo_ambiental'] = df['id_distintivo_ambiental'].map(conversion)

In [57]:
if not df_nuevos_distintivos.empty:
    
    insertar_datos_en_lotes(df_nuevos_distintivos, 'distintivos_ambientales', lote=100)
    
    print(f"{len(df_nuevos_distintivos)} distintivos ambientales nuevos insertados en la base de datos.")
else:
    print("No hay nuevos distintivos ambientales para insertar.")

No hay nuevos distintivos ambientales para insertar.


### Tabla combustibles

In [59]:
tabla_combustibles = extraer_datos('combustibles', columna = '*')
combustibles_guardados = set(tabla_combustibles['nombre_combustible'])

Datos de combustibles extraidos correctamente


In [60]:
nuevos_combustibles = {}
index = tabla_combustibles['id_combustible'].max() + 1 if not tabla_combustibles.empty else 1

for combustible in df['combustible'].unique():
    if pd.notnull(combustible) and combustible not in combustibles_guardados: 
        nuevos_combustibles[index] = combustible
        index += 1

df_nuevos_combustibles = pd.DataFrame.from_dict(nuevos_combustibles, orient='index', columns=['nombre_combustible']).reset_index()
df_nuevos_combustibles.rename(columns={'index': 'id_combustible'}, inplace=True)

df_combustibles_actualizado = pd.concat([tabla_combustibles, df_nuevos_combustibles], ignore_index=True)

conversion = dict(zip(df_combustibles_actualizado['nombre_combustible'], df_combustibles_actualizado['id_combustible']))
df['id_combustible'] = df['combustible'].map(conversion)
df = df.drop(columns=['combustible'])

In [61]:
if not df_nuevos_combustibles.empty:
    
    insertar_datos_en_lotes(df_nuevos_combustibles, 'combustibles', lote=100)
    
    print(f"{len(df_nuevos_combustibles)} combustibles nuevos insertados en la base de datos.")
else:
    print("No hay nuevos combustibles para insertar.")

No hay nuevos combustibles para insertar.


### Tabla tipo_cambio

In [63]:
tabla_tipos_cambio = extraer_datos('tipos_cambio', columna = '*')
tipos_cambio_guardados = set(tabla_tipos_cambio['nombre_tipo_cambio'])

Datos de tipos_cambio extraidos correctamente


In [64]:
nuevos_tipos_cambio = {}
index = tabla_tipos_cambio['id_tipo_cambio'].max() + 1 if not tabla_tipos_cambio.empty else 1

for tipo_cambio in df['tipo_cambio'].unique():
    if pd.notnull(tipo_cambio) and tipo_cambio not in tipos_cambio_guardados: 
        nuevos_tipos_cambio[index] = tipo_cambio
        index += 1

df_nuevos_tipos_cambio = pd.DataFrame.from_dict(nuevos_tipos_cambio, orient='index', columns=['nombre_tipo_cambio']).reset_index()
df_nuevos_tipos_cambio.rename(columns={'index': 'id_tipo_cambio'}, inplace=True)

df_tipos_cambio_actualizado = pd.concat([tabla_tipos_cambio, df_nuevos_tipos_cambio], ignore_index=True)

conversion = dict(zip(df_tipos_cambio_actualizado['nombre_tipo_cambio'], df_tipos_cambio_actualizado['id_tipo_cambio']))
df['id_tipo_cambio'] = df['tipo_cambio'].map(conversion)
df = df.drop(columns=['tipo_cambio'])

In [65]:
if not df_nuevos_tipos_cambio.empty:
    
    insertar_datos_en_lotes(df_nuevos_tipos_cambio, 'tipos_cambio', lote=100)
    
    print(f"{len(df_nuevos_tipos_cambio)} tipos_cambio nuevos insertados en la base de datos.")
else:
    print("No hay nuevos tipos_cambio para insertar.")

No hay nuevos tipos_cambio para insertar.


### Tabla sobrealimentaciones

In [67]:
tabla_sobrealimentaciones = extraer_datos('sobrealimentaciones', columna = '*')
sobrealimentaciones_guardadas = set(tabla_sobrealimentaciones['nombre_sobrealimentacion'])

Datos de sobrealimentaciones extraidos correctamente


In [68]:
nuevas_sobrealimentaciones = {}
index = tabla_sobrealimentaciones['id_sobrealimentacion'].max() + 1 if not tabla_sobrealimentaciones.empty else 1

for sobrealimentacion in df['id_sobrealimentacion'].unique():
    if pd.notnull(sobrealimentacion) and sobrealimentacion not in sobrealimentaciones_guardadas: 
        nuevas_sobrealimentaciones[index] = sobrealimentacion
        index += 1

df_nuevas_sobrealimentaciones = pd.DataFrame.from_dict(nuevas_sobrealimentaciones, orient='index', columns=['nombre_sobrealimentacion']).reset_index()
df_nuevas_sobrealimentaciones.rename(columns={'index': 'id_sobrealimentacion'}, inplace=True)

df_sobrealimentaciones_actualizado = pd.concat([tabla_sobrealimentaciones, df_nuevas_sobrealimentaciones], ignore_index=True)

conversion = dict(zip(df_sobrealimentaciones_actualizado['nombre_sobrealimentacion'], df_sobrealimentaciones_actualizado['id_sobrealimentacion']))
df['id_sobrealimentacion'] = df['id_sobrealimentacion'].map(conversion)

In [69]:
if not df_nuevas_sobrealimentaciones.empty:
    
    insertar_datos_en_lotes(df_nuevas_sobrealimentaciones, 'sobrealimentaciones', lote=100)
    
    print(f"{len(df_nuevas_sobrealimentaciones)} sobrealimentaciones nuevas insertados en la base de datos.")
else:
    print("No hay nuevas sobrealimentaciones para insertar.")

No hay nuevas sobrealimentaciones para insertar.


### Tabla tipo_traccion

In [71]:
tabla_tipo_traccion = extraer_datos('tipo_traccion', columna = '*')
tipo_traccion_guardados = set(tabla_tipo_traccion['nombre_traccion'])

Datos de tipo_traccion extraidos correctamente


In [72]:
nuevos_tipo_traccion = {}
index = tabla_tipo_traccion['id_traccion'].max() + 1 if not tabla_tipo_traccion.empty else 1

for tipo_traccion in df['id_traccion'].unique():
    if pd.notnull(tipo_traccion) and tipo_traccion not in tipo_traccion_guardados: 
        nuevos_tipo_traccion[index] = tipo_traccion
        index += 1

df_nuevos_tipo_traccion = pd.DataFrame.from_dict(nuevos_tipo_traccion, orient='index', columns=['nombre_traccion']).reset_index()
df_nuevos_tipo_traccion.rename(columns={'index': 'id_traccion'}, inplace=True)

df_tipo_traccion_actualizado = pd.concat([tabla_tipo_traccion, df_nuevos_tipo_traccion], ignore_index=True)

conversion = dict(zip(df_tipo_traccion_actualizado['nombre_traccion'], df_tipo_traccion_actualizado['id_traccion']))
df['id_traccion'] = df['id_traccion'].map(conversion)

In [73]:
if not df_nuevos_tipo_traccion.empty:
    
    insertar_datos_en_lotes(df_nuevos_tipo_traccion, 'tipo_traccion', lote=100)
    
    print(f"{len(df_nuevos_tipo_traccion)} tipo_traccion nuevos insertados en la base de datos.")
else:
    print("No hay nuevos tipo_traccion para insertar.")

No hay nuevos tipo_traccion para insertar.


## Subir registros prestaciones

In [75]:
# Mapeo para las marcas en la tabla prestaciones

tabla_marcas = extraer_datos('marcas', columna = '*')
tabla_marcas['nombre_marca'] = tabla_marcas['nombre_marca'].str.upper()
conversion = dict(zip(tabla_marcas['nombre_marca'], tabla_marcas['id_marca']))
df['id_marca'] = df['id_marca'].map(conversion)

Datos de marcas extraidos correctamente


In [76]:
# Mapeo para los links en la tabla prestaciones

tabla_links = extraer_datos('links_coches', columna = 'id_coche, link_anuncio')
conversion = dict(zip(tabla_links['link_anuncio'], tabla_links['id_coche']))
df['id_coche'] = df['link_coche'].map(conversion)
df = df.drop(columns=['link_coche'])

Datos de links_coches extraidos correctamente


In [77]:
df['fecha_extraccion'] = pd.to_datetime(df['fecha_extraccion'], format='%d/%m/%Y', errors='coerce' )
df['fecha_extraccion'] = df['fecha_extraccion'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
# Comprobamos que los coches que vamos a subir no están ya en la tabla
df_prestaciones = extraer_datos('prestaciones')
df = df[df_prestaciones.columns]

coches_completos = pd.concat([df_prestaciones, df], ignore_index = True)

columnas_para_duplicados = [col for col in coches_completos.columns if col not in ['fecha_extraccion', 'link_coche']]
coches_completos = coches_completos.drop_duplicates(subset=columnas_para_duplicados, keep='first')

df_datos_nuevos = coches_completos.iloc[len(df_prestaciones):]

print(f"Número de coches nuevos: {len(df_datos_nuevos)}")

Datos de prestaciones extraidos correctamente


In [78]:
#Convertir a nan a none
for columna in df.columns:
    df[columna] = df[columna].replace([np.nan, pd.NA], None)

In [79]:
insertar_datos_en_lotes(df_datos_nuevos, 'prestaciones', lote=100)

Datos insertados en prestaciones correctamente.
